In [1]:
from tqdm import tqdm
import torch
from torch import Tensor
import numpy as np
from jaxtyping import Int, Float
from typing import List, Optional, Tuple
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pickle

import pandas as pd
from tqdm import tqdm
from einops import repeat
import multiprocessing
import sys

from DSA.dmd import DMD
from DSA.simdist import SimilarityTransformDist
from sklearn.manifold import MDS

sys.path.append('../')
from models import get_model_from_name
from data import get_data_sample
from activations import get_residual_stream_activations_for_layer_wise_dmd, get_residual_stream_activations
from dmd import apply_dmd

torch.set_grad_enabled(True)

# List all available GPUs
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    print(f"Current CUDA device: {torch.cuda.current_device()}")

else:
    print("CUDA is not available. Listing CPUs instead.")
    print(multiprocessing.cpu_count())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jsalle/.conda/envs/p11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is not available. Listing CPUs instead.
48


In [2]:
model_names = ['gpt2-small', 'pythia-1b', 'pythia-2.8b']

for model_name in model_names:
    model, tokenizer = get_model_from_name(model_name)
    x, wikitext_sample = get_residual_stream_activations(model, tokenizer, seed=41)
    with open(f'../data/{model_name}_wikitext103_activations_2.pkl', 'wb') as file: 
        pickle.dump(x, file)

['= Grammy Award for Video of the Year =\nThe Grammy Award for Video of the Year was an honor presented to recording artists at the 24th Grammy Awards in 1982 and the 25th Grammy Awards in 1983 for music videos. The Grammy Awards, an annual ceremony that was established in 1958 and originally called the Gramophone Awards, are presented by the National Academy of Recording Arts and Sciences of the United States to "honor artistic achievement, technical proficiency and overall excellence in the recording industry, without regard to album sales or chart position".\nFirst presented in 1982 to Michael Nesmith for Elephant Parts, the award was reserved for quality "video cassettes or discs in any format created specifically for the home video market". In 1983, Olivia Newton-John was presented the award for Olivia Physical. Beginning with the 1984 Grammy Awards, the Video of the Year award category was replaced with the categories Best Video, Short Form and Best Video Album (now known as Best

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['= Grammy Award for Video of the Year =\nThe Grammy Award for Video of the Year was an honor presented to recording artists at the 24th Grammy Awards in 1982 and the 25th Grammy Awards in 1983 for music videos. The Grammy Awards, an annual ceremony that was established in 1958 and originally called the Gramophone Awards, are presented by the National Academy of Recording Arts and Sciences of the United States to "honor artistic achievement, technical proficiency and overall excellence in the recording industry, without regard to album sales or chart position".\nFirst presented in 1982 to Michael Nesmith for Elephant Parts, the award was reserved for quality "video cassettes or discs in any format created specifically for the home video market". In 1983, Olivia Newton-John was presented the award for Olivia Physical. Beginning with the 1984 Grammy Awards, the Video of the Year award category was replaced with the categories Best Video, Short Form and Best Video Album (now known as Best

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['= Grammy Award for Video of the Year =\nThe Grammy Award for Video of the Year was an honor presented to recording artists at the 24th Grammy Awards in 1982 and the 25th Grammy Awards in 1983 for music videos. The Grammy Awards, an annual ceremony that was established in 1958 and originally called the Gramophone Awards, are presented by the National Academy of Recording Arts and Sciences of the United States to "honor artistic achievement, technical proficiency and overall excellence in the recording industry, without regard to album sales or chart position".\nFirst presented in 1982 to Michael Nesmith for Elephant Parts, the award was reserved for quality "video cassettes or discs in any format created specifically for the home video market". In 1983, Olivia Newton-John was presented the award for Olivia Physical. Beginning with the 1984 Grammy Awards, the Video of the Year award category was replaced with the categories Best Video, Short Form and Best Video Album (now known as Best

In [9]:
# Load file 
model_name = 'pythia-1b'
with open(f'../data/{model_name}_wikitext103_activations.pkl', 'rb') as file: 
    activations = pickle.load(file)